## A stateless oracle (2): the oracle coin and oracle accounts
#### 09.2 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-02-15
* Part 2: Create the Oracle Coin
* Parts 1-4 are only relevant if you want to **create** an Oracle
* Only parts 5-6 are needed to **use** the oracle.

## Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the Purestake credentials

In [1]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [2]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.future.transaction import LogicSig

import algosdk.error
import json
import base64
import hashlib

In [3]:
from pyteal import *

In [4]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

In [5]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

VK6CCXY4IFHIJAVMRVS543LJQEQKOJO6YQ4DZNV3D2XJI4ETYBN5354EQU
CPUT3Z5CI3XOIZ4ARSGUFQD7V4YGYJW5BFAZMXX5YOV4KJCKI6MBCDY5XM
BY5K2AYO7R3G66ICY6SJ2JFVLRMIX677EAEEKDBTJZGP6Q4JVNZRDXDBKA


In [6]:
import json
import requests
import pandas as pd
import numpy as np
import time

## Create Oracle Coin and Oracle Accounts

Alice wants to provide an oracle for the USD/ALGO exchange rate on the Algorand chain. The price will be encoded in the holdings of two accounts. First, she creates the oracle coin. This coin will not be used to *pay* for the oracle, in fact it will never go into circulation. The holdings of the coin will be used as information on the price.

* Total of 1'000 Oracle coins exist, with 6 decimals
* Oracle coin is held by only 2 accounts: `Price` and `Reserve`
* Number of coins in `Price` is equal to the USD/ALGO exchange rate
    * Example: ALGO price is 0.82 Dollars
    * `Price` holds 0.82 Oracle Coins (=82'000 micro-Oracles)
    * As the `Price` holds all "free" coins, **the free float is equal to the exchange rate**
    * Spare Coins held by `Reserve`, in our example 999.18 Coins

### Step 1: Create of Oracle coin

In [ ]:
# Step 1: prepare AssetConfigTxn
sp = algod_client.suggested_params()
token_supply = 1000                              # Token supply (big units)
token_decimals =  6                              # Digits after the comma
token_total = token_supply * 10**token_decimals  # Specify number of SMALLER unit ("cents")

token_name  = "USD/Algo Oracle coin"
token_url   = "https://www.coingecko.com/en/coins/algorand"
token_unit  = "USDALGO"

txn = AssetConfigTxn(
    sender=Alice['public'],                    # Creator of the ASA
    sp=sp,                      
    total=token_total,                         # Total supply in SMALL unit
    decimals=token_decimals,                   # Decimals
    default_frozen=False,                      # Are tokens frozen by default?
    unit_name=token_unit,                      # Abbreviation     
    asset_name=token_name,                     # Name
    url=token_url,                             # URL
    manager=Alice['public'],                   # Special roles (later more)
    reserve=Alice['public'],                   # Special roles
    freeze=Alice['public'],                    # Special roles
    clawback=Alice['public']                   # Special roles
)

### Step 2+3+4: Sign, send, wait
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)
txinfo = wait_for_confirmation(algod_client,txid)

In [ ]:
# Step 5:  Get the asset ID
oracle_id = txinfo['asset-index']
print(oracle_id)

In [ ]:
# Create a link
print('https://testnet.algoexplorer.io/asset/{}'.format(oracle_id))

### Step 2: Create two accounts `Price` and `Reserve`

#### Transactions with "Price" and "Reserve"
* We are going to execute two types of transactions with the over/under accounts
    1. Technical operations of the oracle
        * Opt-in
        * Update the holdings
    2. Transactions where the oracle is used

#### The idea of a signed smart signature
* For (1), it is clear that we prefer a Smart Signature, so that anybody can use the Oracle.
* For (2), having to implement all cases of opt-in and updating is a hassle in a Smart Signature.
* For this reason, we create a signed smart signature
    * First we create two simple accounts `Price` and `Reserve`
    * `Price` and `Reserve` can sign special operations such as opt-in – case (1)
    * In the next notebook, we crate a signed Smart Signature for the oracle transactions – case (2)

### 2.1 Create accounts
* Create two new accounts from scratch
* Save credentials to hard drive for deployment to the Linux server later

In [24]:
Price = generate_account_dict()
Reserve = generate_account_dict()

In [ ]:
print(Price['public'])
print(Reserve['public'])

In [33]:
# Credentials for Linux server; save to hard drive
cred_oracle = {"Price": Price,
               "Reserve": Reserve, 
               "purestake_token": cred['purestake_token'], 
               "algod_test": cred['algod_test'], 
               "algod_main": cred['algod_main'],
               "oracle_id": oracle_id}

cred_json = json.dumps(cred_oracle,indent=4)

filename = 'credentials_oracle'           # saved in SAME folder as this notebook
with open(filename, 'w') as outfile:      # option 'w' ensures overwriting of existing file
    outfile.write(cred_json)

### 2.2 Funding and Opt-in transactions

#### Funding
* New accounts must be funded so that they can pay for TX cost and have minimum holdings
* Add some more funding for future TX cost

In [41]:
for receiver in [Price, Reserve]:
    # Step 1: prepare
    sp = algod_client.suggested_params()
    amt = int(1e6)
    txn = PaymentTxn(sender = Alice['public'],sp=sp,
                     receiver=receiver['public'],amt=amt)                               
    # Step 2+3+4: sign, send, wait for ...
    stxn = txn.sign(Alice['private'])
    txid = algod_client.send_transaction(stxn)
    txinfo = wait_for_confirmation(algod_client,txid)

Current round is  20362479.
Waiting for round 20362479 to finish.
Waiting for round 20362480 to finish.
Transaction CJMPGBHQOEAGFBRTGBNDTGABV3WBG7BNC4GTEDAPG2T4QQEYFMSA confirmed in round 20362481.
Current round is  20362481.
Waiting for round 20362481 to finish.
Waiting for round 20362482 to finish.
Transaction AEKWS3JAKSURV2NFSX6EHQ64MLVHSMS2FXHT5AO3VTZSG2I4G2WQ confirmed in round 20362483.


#### Opt into Oracle Coin
* Standard opt-in transaction (transfer zero coins to yourself)
* Organised as a for-loop for efficiency and elegance

In [43]:
for account in [Price, Reserve]:
    # Step 1: prepare
    sp = algod_client.suggested_params()          # Suggested params
    amt = int(0)                                  # <-- Send 0 coins for opt in

    txn = AssetTransferTxn(
        sender=account['public'],                 # <-- from account ...
        sp=sp,
        receiver=account['public'],               # <-- ... to self
        amt=amt,                                  # <-- Zero coins
        index=oracle_id                           # <-- Specify which ASA
    )                               

    # Step 2+3+4: sign, send, wait for ...
    stxn = txn.sign(account['private'])
    txid = algod_client.send_transaction(stxn)
    txinfo = wait_for_confirmation(algod_client,txid)

Current round is  20362509.
Waiting for round 20362509 to finish.
Waiting for round 20362510 to finish.
Transaction NBZHJNVM2JT6STZCW5UWNHODAT4IOK5HBZQ3TDRF2IWG7NOLHE6Q confirmed in round 20362511.
Current round is  20362511.
Waiting for round 20362511 to finish.
Waiting for round 20362512 to finish.
Transaction ALV5NTKCK4THMEU7ZXWZZPHM56P3ACZELXSPZP3SQVFXK6SCXNYQ confirmed in round 20362513.


### 2.3 Transfer
* Alice transfers all coins to the Reserve account
* This corresponds to a, oracle price of zero
* The correct balances will be established in the 09.3 notebook

In [ ]:
# Step 1: prepare
sp = algod_client.suggested_params()          # Suggested params

txn = AssetTransferTxn(
    sender=Alice['public'], 
    sp=sp,
    receiver=Reserve['public'], 
    amt=int(token_total),                     # 1000 coins in small units
    index=oracle_id 
)                               

# Step 2+3+4: sign, send, wait for ...
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)
txinfo = wait_for_confirmation(algod_client,txid)

### 2.3 Alice designates `Reserve` as the Reserve for the oracle coin
* This small step makes it possible to read the current oracle price as "circulating supply" on algoexplorer
    * Why? Because the only account other from Reserve that holds oracle coins is the Price account

In [9]:
# Step 1: prepare AssetConfigTxn
sp = algod_client.suggested_params()

txn = AssetConfigTxn(
    sender=Alice['public'],                   # Creator of the ASA
    sp=sp, 
    index=oracle_id,     
    manager=Alice['public'],                   
    reserve=Reserve['public'],                # <--- new reserve
    freeze=Alice['public'],                    
    clawback=Alice['public']                   
)

### Step 2+3+4: Sign, send, wait
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)
txinfo = wait_for_confirmation(algod_client,txid)

Current round is  20421491.
Waiting for round 20421491 to finish.
Waiting for round 20421492 to finish.
Transaction 2FCJ7A5YPLPFBNCCZHUN27EX2EIRS2DAERC6H3OBWQ2WGUSLRGFA confirmed in round 20421493.


## Updated information about the oracle coin
* Circulating supply corresponds to current price
* At the moment, this is zero

In [76]:
print('https://testnet.algoexplorer.io/asset/{}'.format(oracle_id))

https://testnet.algoexplorer.io/asset/77534697
